In [ ]:
from dataLoader import *
import basePlots as bp
import combinedPlots as cbp
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as scistats
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
EXPERIMENT_14_11_file = "../src/build/csv/experiment-14-11-2025.csv"
exp_14_11 =  Experiment(EXPERIMENT_14_11_file)
_filter = Filter(exp_14_11.inner_df)

In [ ]:
_filter.set_x("repetitions")
_filter.set_y("repetitions_duration",explode=True)
_filter.set_hue("version",used=True)
_filter.set_default("kernel","Bilateral")
_filter.set_default("repetitions",1000)
_filter.set_subsets("version",["ReferenceBilateral","DGBilateral"])
bp.compare_cdf(exp_14_11,_filter)

# CDF distances

## K-S Test
[K-S Test](https://en.wikipedia.org/wiki/Kolmogorov–Smirnov_test), Max of the distance point to point, but for every point of the ECDF1 and ECDF2
Tells just how much two distributions are different

Returns how much the distributions differ at the max points.

Returning 0.05 means that at most there is 5% probability difference.

(How different are ECDF1 and ECDF2)

## Total Variation
[Total Variation](https://en.wikipedia.org/wiki/Total_variation_distance_of_probability_measures)
The total area between two distributions
Returns miliseconds

like how much across all the probabilistic space one is quicker then the other. Like a mean ~

## CVM Cramér–von Mises criterion
[C-M Criterion](https://en.wikipedia.org/wiki/Cramér–von_Mises_criterion)

[scipy imp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.cramervonmises_2samp.html)

Compares the interleaving of the distributions, so is only based on how each sample ranks together, and doesnt care of the output, is only statistical and do not depend on what is measured

## Probability of Superiority
P( A < B), How much A is better then B,
In our case, whats the probability that A is quicker then B 

In [ ]:
a = Ecdf([1,2,2,2,8,3,5,6,3,8,2,13,4,5,12])
b = Ecdf([1,3,9,8,1,1,2,2,3,3,4,5,6,7,7,6])

In [ ]:
EcdfMetrics.k_s_test(b,a)

In [ ]:
EcdfMetrics.ecdf_area(a,b)

In [ ]:
a.plot()
b.plot()

In [ ]:
_filter.set_x(used=False)
_filter.set_y(explode=False)
_filter.set_default("work_size",4)
df = exp_14_11.filter(_filter)

In [ ]:
print(_filter)

In [ ]:
ecdfs = []
for index,row in df.iterrows():
    ecdfs.append(Ecdf(row["repetitions_duration"],row["version"]))

In [ ]:
for ecdf in ecdfs:
    ecdf.plot()
plt.legend()

In [ ]:
ks_val = EcdfMetrics.k_s_test(ecdfs[1],ecdfs[0])
total_area,real_area = EcdfMetrics.ecdf_area(ecdfs[1],ecdfs[0])
cvm_stat = EcdfMetrics.cvm_distance(ecdfs[1],ecdfs[0])
print(f"KS : {ks_val}, total_area : {total_area}, real_area : {real_area}, cvm_stat: {cvm_stat}")

In [ ]:
EcdfMetrics.prob_superiority(ecdfs[0],ecdfs[1])

In [ ]:
df.columns

In [ ]:
variables = ["repetitions","warmups","work_size","flush_l2","blocking","kernel","version"]
measures = ["ks","total_area","signed_area","cvm","superiority"]

In [ ]:
exp1 = Experiment(source_csv_path="../src/build/csv/exp-18-11.csv",actions="",cache=False)
exp2 = Experiment(source_csv_path="../src/build/csv/exp-17-11-1.csv",actions="",cache=False)
exp_ecdf = Experiment.compare_experiments_ecdf(exp1,exp2)


In [ ]:
ranked_df = exp_ecdf.inner_df.sort_values(by="repetitions_duration_mean_difference",ascending=False)
ranked_df[ranked_df["version"].str.contains("Reference")][["repetitions","warmups","work_size","flush_l2","blocking","kernel","version","repetitions_duration_mean_difference",EcdfMetrics.ECDF_COMPARISON_SCORE]]

In [ ]:
from scipy import stats
dfnew = df[df["kernel"] == "Accuracy"]
dfnew = dfnew[dfnew["version"] == "ReferenceAccuracy"]

In [ ]:
print(_filter)

In [ ]:
_filter = Filter.from_best_config(exp_ecdf.inner_df)
bp.compare_variables_impact_on_ecdf_metrics(exp_ecdf,_filter)

In [ ]:
_filter = Filter.from_best_config(exp_ecdf.inner_df)
_filter.set_x("repetitions")
_filter.set_y(EcdfMetrics.ECDF_COMPARISON_SCORE,explode=False)
_filter.set_subsets("version","Reference")
bp.compare_configurations_ecdf_metrics(exp_ecdf,_filter)

In [ ]:
_filter = Filter.from_best_config(exp_ecdf.inner_df)
_filter.set_x("repetitions")
_filter.set_y("repetitions_duration_mean_difference",explode=False)
_filter.set_subsets("version","Reference")
bp.compare_configurations_ecdf_metrics(exp_ecdf,_filter)

In [ ]:
_filter = Filter.from_best_config(exp_ecdf.inner_df)
_filter.set_x("repetitions")
_filter.set_y("superiority")
_filter.set_default("version","ReferenceBilateral")
_filter.set_hue("version",used=True)
_filter.set_subsets("version",["ReferenceBilateral","DGBilateral"])
bp.line_plot(exp_ecdf,_filter)

In [ ]:
_filter = Filter.from_best_config(exp_ecdf.inner_df)
_filter.set_x("repetitions")
_filter.set_default("version","ReferenceBilateral")
_filter.set_hue("version",used=True)
_filter.set_subsets("version",["ReferenceBilateral","DGBilateral"])
cbp.combined_ecdf_scores(exp_ecdf,_filter)

In [ ]:
_filter = Filter.from_best_config(exp_ecdf.inner_df)
_filter.set_x("repetitions")
_filter.set_hue("work_size",used=True)
#_filter.set_subsets("version",["ReferenceBilateral","DGBilateral"])
cbp.combined_ecdf_scores(exp_ecdf,_filter)